## Подготовка данных для задания

In [ ]:
#!pip install pymorphy2

In [2]:
import re

import pandas as pd
import pymorphy2

text_df = pd.read_csv("content_description.csv", sep='\t')
#text_df = pd.read_csv("./data/content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)

# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
29,0,и,7
37,0,который,5
111,0,шерлок,4
77,0,с,4
84,0,сериал,4
75,0,риколетти,3
31,0,история,2
21,0,для,2
49,0,о,2


# 12.3  Домашняя работа

12.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [160]:
# выведите на экран топ-10 слов

# приводим двухмерный список с повторяющимися элементами
# к одномерному списку без повторений
word_list = list(set(sum(normalized_corpus, [])))

# готовим словарь со словами из word_list
word_dict = {'words': word_list, 'freq': [], 'tag': []}
for word in word_list:

    # для каждого слова определяем часть речи
    word_dict['tag'].append(morph.parse(word)[0].tag.POS)

    # считаем процент документов, где встречается слово
    count = 0
    for description in normalized_corpus:
        if word in description:
            count +=1
    word_dict['freq'].append(int(count / number_of_documents * 100))

# составляем список с лишними тегами
useless_tags = ['CONJ', 'PREP', 'PRCL', 'NPRO', 'ADJF']

# Датасет с неинформативными словами
raw_doc_frequency = pd.DataFrame(word_dict).sort_values('freq', ascending=False)

# очищенный датасет
doc_frequency = raw_doc_frequency[~raw_doc_frequency['tag'].isin(useless_tags)]
doc_frequency.head(10)


,words,freq,tag
437,онлайн,75,ADVB
269,год,62,NOUN
110,смотреть,62,INFN
601,можно,37,PRED
174,жизнь,37,NOUN
178,режиссёр,37,NOUN
575,история,37,NOUN
393,рождественский,37,NOUN
80,исполнить,37,INFN
372,жертва,37,NOUN


12.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [247]:
raw_text = text_df.values[3][1]

In [151]:
# запишите необходимое регулярное выражение в переменную reg_expr 
reg_expr = r'[А-Я]\w+.[А-Я]\w+'

# скомпилированное выражение добавьте в обьёект reg_expr_compiled
reg_expr_compiled = re.compile(reg_expr)

# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):

    # эта часть убирает сочетания типа "Не_имя_с_заглавной_буквы Имя"
    is_name = True
    for word in g.split():
        if morph.parse(word)[0].tag.POS != 'NOUN':
            is_name = False
    if is_name:
        print(g)

Стивена Фрая
Джоном Дженксом
Теда Уоллеса


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`, сохранить в переменнуб `genres_df`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [246]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

# разворот словаря genre_dict и создание датафрейма
new_genre_dict = dict(chain.from_iterable(zip(v, repeat(k)) for k, v in genre_dict.items()))
genres_df = pd.DataFrame({'word': new_genre_dict.keys(), 'genre': new_genre_dict.values()})

# слияние датафреймов с жанрами и id фильмов
merged_df = genres_df.merge(word_count_df, left_on='word', right_on='word').sort_values('doc_id')
merged_df = merged_df[['doc_id', 'genre']].drop_duplicates()

# группировка жанров по id фильма
merged_df.groupby('doc_id').apply(lambda x: ', '.join(x['genre'])).reset_index(name='genre')

,doc_id,genre
0,0,"детектив, триллер"
1,1,сказка
2,2,мелодрама
3,3,комедия
4,4,комедия
5,6,триллер
